### 데이터 셋업

In [ ]:
# ======1단계: 입력 폴더 경로 설정
path_in = r"..\assets\ply\impeller\3wings"   # Windows 경로 예시
downsample_N = 20000         # 각각 포인트 샘플링 수


In [ ]:
import os
import numpy as np
import open3d as o3d
from pathlib import Path

#======= 2단계: 단일 PLY 파일 → 점군으로 변환 후 .xyz 저장 =======
def convert_to_xyz(path_in: str, downsample_N: int = 20000):
    """
    단일 PLY 파일 → 점군으로 변환 후 .xyz 저장
    """
    mesh = o3d.io.read_triangle_mesh(path_in)
    if mesh.has_triangles():
        pcd = mesh.sample_points_poisson_disk(number_of_points=downsample_N)
    else:
        pcd = o3d.io.read_point_cloud(path_in)
        if len(pcd.points) > downsample_N:
            pts = np.asarray(pcd.points)
            idx = np.random.choice(len(pts), downsample_N, replace=False)
            pcd.points = o3d.utility.Vector3dVector(pts[idx])

    out_path = Path(path_in).with_name(Path(path_in).stem + "_point.xyz")
    np.savetxt(out_path, np.asarray(pcd.points), fmt="%.6f")
    print(f"[OK] {path_in} → {out_path} (N={len(pcd.points)})")


# 지정한 폴더에 있는 모든 .ply 파일 변환
folder = Path(path_in)
if not folder.is_dir():
    print(f"[ERR] 폴더 경로가 아님: {folder}")
else:
    ply_files = list(folder.glob("*.ply"))
    if not ply_files:
        print(f"[INFO] {folder} 안에 .ply 파일이 없습니다.")
    else:
        for ply in ply_files:
            convert_to_xyz(str(ply), downsample_N=downsample_N)
        print(f"[DONE] 총 {len(ply_files)} 개 파일 변환 완료.")


In [ ]:
import numpy as np
import re
from pathlib import Path

# ====== 3단계: .xyz 파일 → .xyzc 파일로 변환 ======
def convert_xyz_to_xyzc(xyz_path: Path):
    """
    파일명에서 숫자 추출 → 라벨로 추가 → .xyzc 저장
    예: impeller_b3_010_point.xyz → 라벨 = 10.00000000
    """
    # 파일 이름에서 숫자 추출
    m = re.search(r"_(\d+)", xyz_path.stem)
    if not m:
        print(f"[SKIP] 라벨 숫자 추출 실패: {xyz_path.name}")
        return None
    
    label_value = float(m.group(1))  # "010" → 10.0

    # xyz 불러오기
    xyz = np.loadtxt(xyz_path)
    if xyz.ndim == 1:
        xyz = xyz.reshape(-1, 3)

    # 라벨 추가
    label_column = np.full((xyz.shape[0], 1), label_value)
    xyzc = np.hstack((xyz, label_column))

    # 저장
    out_path = xyz_path.with_suffix(".xyzc")
    np.savetxt(out_path, xyzc, fmt="%.6f %.6f %.6f %.8f")
    print(f"[OK] {xyz_path.name} → {out_path.name} (label={label_value:.8f})")
    return out_path


# ====== 폴더 내 모든 _point.xyz 파일 처리 ======
folder = Path(path_in)
xyz_files = list(folder.glob("*_point.xyz"))

if not xyz_files:
    print(f"[INFO] {folder} 안에 '_point.xyz' 파일이 없습니다.")
else:
    for xyz_file in xyz_files:
        convert_xyz_to_xyzc(xyz_file)

    print(f"[DONE] 총 {len(xyz_files)} 개 파일 변환 완료.")


In [ ]:
import numpy as np
from pathlib import Path

# ====== 4단계: 모든 XYZC 파일 병합 ======
path_in = r"..\assets\ply"   # .xyzc 파일들이 있는 폴더 경로
output_file = Path(path_in) / "merged_all.xyzc"  # 최종 합쳐진 파일명

# 폴더 내 모든 .xyzc 찾기
folder = Path(path_in)
xyzc_files = list(folder.glob("*.xyzc"))

if not xyzc_files:
    print(f"[INFO] {folder} 안에 .xyzc 파일이 없습니다.")
else:
    merged_data = []
    for f in xyzc_files:
        try:
            data = np.loadtxt(f)
            if data.ndim == 1:  # 한 줄짜리 방지
                data = data.reshape(1, -1)
            merged_data.append(data)
            print(f"[READ] {f.name} (N={len(data)})")
        except Exception as e:
            print(f"[ERR] {f.name} 불러오기 실패: {e}")

    if merged_data:
        merged_data = np.vstack(merged_data)
        np.savetxt(output_file, merged_data, fmt="%.6f %.6f %.6f %.8f")
        print(f"[DONE] {len(xyzc_files)} 개 파일 합침 → {output_file} (총 {len(merged_data)} 포인트)")
    else:
        print("[INFO] 합칠 데이터가 없습니다.")
